In [1]:
import pandas as pd
import numpy as np

In [2]:
cars = pd.read_csv('datasets/cars_processed_new.csv')
cars.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Origin,Age
0,18.000000,8,307.0,130,3504,12.0,US,49
1,16.000000,8,304.0,150,3433,12.0,US,49
2,17.000000,8,302.0,140,3449,10.5,US,49
3,14.000000,8,454.0,220,4354,9.0,US,49
4,23.551429,8,440.0,215,4312,8.5,US,49


In [3]:
x = cars.drop(['MPG', 'Origin'], axis =1)
y = cars['MPG']


In [4]:
from sklearn.linear_model import Lasso

In [5]:
lasso = Lasso(alpha=.8)
lasso.fit(x,y)

Lasso(alpha=0.8, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [8]:
predictors = x.columns
coef = pd.Series(lasso.coef_, predictors).sort_values()
print(coef)

Age            -0.673784
Weight         -0.006603
Horsepower     -0.004511
Cylinders      -0.000000
Acceleration    0.000000
Displacement    0.001366
dtype: float64


In [9]:
# Important predictors are close to 0

lasso_features = ['Age', 'Weight']
x[lasso_features].head()

,Age,Weight
0,49,3504
1,49,3433
2,49,3449
3,49,4354
4,49,4312


In [10]:
from sklearn.tree import DecisionTreeRegressor

In [11]:
tree = DecisionTreeRegressor(max_depth=4)
tree.fit(x,y)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=4,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [13]:
predictors = x.columns

coef = pd.Series(tree.feature_importances_, predictors).sort_values()
print(coef)

Cylinders       0.000000
Acceleration    0.000000
Weight          0.058285
Age             0.112716
Horsepower      0.183409
Displacement    0.645590
dtype: float64


In [14]:
tree_features = ['Horsepower', 'Displacement']

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

def build_model(x, y, test_frac):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_frac)
    model = LinearRegression(normalize=True).fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print('R2 Score :', r2_score(y_test, y_pred))

In [17]:
build_model(x[lasso_features], y, test_frac=0.2)

R2 Score : 0.7140706688945989


In [18]:
build_model(x[tree_features], y, test_frac=0.2)

R2 Score : 0.5745219053039391
